In [1]:
import numpy as np


In [1]:
import os.path as osp
import cv2
import numpy as np

import sys
sys.path.append('../../../') 

from dust3r.datasets.base.base_stereo_view_dataset import BaseStereoViewDataset
from dust3r.utils.image import imread_cv2


In [4]:
from itertools import combinations
from collections import defaultdict
from tqdm.auto import tqdm
from scipy.spatial import cKDTree
from sklearn.neighbors import NearestNeighbors

def compute_iou(pointcloud1, pointcloud2, pixels_count, distance_threshold=0.4):
    nbrs = NearestNeighbors(n_neighbors=1, algorithm = 'kd_tree').fit(pointcloud2)
    distances, indices = nbrs.kneighbors(pointcloud1)
    intersection1 = np.count_nonzero(distances.flatten() < distance_threshold)
    
    # nbrs = NearestNeighbors(n_neighbors=1, algorithm = 'kd_tree').fit(pointcloud1)
    # distances, indices = nbrs.kneighbors(pointcloud2)
    # intersection2 = np.count_nonzero(distances.flatten() < distance_threshold)
    
    # iou = min(intersection1 / pixels_count, intersection2 / pixels_count)
    iou = intersection1 / len(pointcloud1)
    print('intersection1', intersection1)
    print('iouuuuuuuuu', iou)
    if iou > 0.75:
        print('KEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEek')
        # return 0.0

    return intersection1

def simulate_occlusion(point_cloud, radius):
    """
    Simulates occlusion by attaching a ball of fixed radius to each 3D point.
    Points occluded by another point within the radius are removed.
    """
    if len(point_cloud.shape) == 3:
        point_cloud = point_cloud.reshape(-1, 3)
    return point_cloud
    # tree = cKDTree(point_cloud)
    # 
    # indices = tree.query_ball_tree(tree, radius)
    # 
    # visible_mask = np.full(len(point_cloud), True, dtype=bool)
    # for i, neighbors in enumerate(indices):
    #     if visible_mask[i]:
    #         for neighbor in neighbors[1:]:
    #             visible_mask[neighbor] = False
    # 
    # visible_points = point_cloud[visible_mask]
    # return visible_points

def compute_iou_with_occlusion(pc1, pc2, radius=0.0005):
    """
    Computes the IoU between two 3D point clouds with artificial occlusion.
    """
    visible_pc1 = simulate_occlusion(pc1, radius)
    visible_pc2 = simulate_occlusion(pc2, radius)
    
    print('visible_pc1 and 2', len(visible_pc1), len(visible_pc2))
    
    print(pc1.shape[0], pc1.shape[1], pc1.shape[2])
    
    iou = compute_iou(visible_pc1, visible_pc2, pixels_count=pc1.shape[0] * pc1.shape[1])
    
    if iou == 0:
        return 0
    
    print('iou',iou)
    return iou

def quality_pair_score(iou, alpha):
    """
    Compute the quality pair score s = IoU × 4 cos(α)(1 - cos(α))
    """
    angle_term = 4 * np.cos(alpha) * (1 - np.cos(alpha))
    return iou * angle_term if angle_term > 0 else 0

def select_best_pairs(dataset, iou_threshold=0.75, score_threshold=0.1, pairs_per_image=5, pairs_number=10):
    """
    Select the best image pairs using a greedy algorithm
    """
    # pairs = list(combinations(dataset, 2))
    pair_scores = []

    for i in range(100):
        img1 = dataset[i][0]
        img2 = dataset[i][1]
        
        print(img1['idx'], img2['idx'])
        
        iou = compute_iou_with_occlusion(img1['pts3d'], img2['pts3d'])
        if iou == 0:
            continue
        
        pose1, pose2 = img1['camera_pose'], img2['camera_pose']
        rotation_diff = pose1[:3, :3].T @ pose2[:3, :3]  # Relative rotation matrix
        alpha = np.arccos(np.clip((np.trace(rotation_diff) - 1) / 2, -1, 1))  # Angle in radians
        print(alpha)
        score = quality_pair_score(iou, alpha)
        print('SCORE', img1['idx'], img2['idx'], score)
        if score > score_threshold:
            pair_scores.append((score, img1, img2))

    pair_scores.sort(key=lambda x: x[0], reverse=True)

    selected_pairs = []
    used_images = set()

    for score, img1, img2 in pair_scores:
        if img1['idx'] in used_images or img2['idx'] in used_images:
            continue

        selected_pairs.append((img1, img2))
        used_images.add(img1['idx'])
        used_images.add(img2['idx'])

        for img in dataset:
            if img['idx'] not in used_images:
                if compute_iou(img['pts3d'], img1['pts3d']) > iou_threshold or \
                   compute_iou(img['pts3d'], img2['pts3d']) > iou_threshold:
                    used_images.add(img['idx'])
                    
    # used_images = defaultdict(int)
    # 
    # for score, img1, img2 in pair_scores:
    #     if used_images[img1['idx']] > pairs_per_image or used_images[img2['idx']] > pairs_per_image:
    #         continue
    # 
    #     selected_pairs.append((img1, img2))
    #     used_images[img1['idx']] += 1
    #     used_images[img2['idx']] += 1

    # if len(selected_pairs) < pairs_number: 
    #     for i in range(len(dataset) - 1):
    #         if len(selected_pairs) >= pairs_number:
    #             break
    #         img1, img2 = dataset[i], dataset[i + 1]
    #         selected_pairs.append((img1, img2))

    return selected_pairs

selected_pairs = select_best_pairs(dataset)
for img1, img2 in selected_pairs:
    print(f"Selected pair: Image {img1['idx']} and Image {img2['idx']}")

(0, 0, 0) (0, 0, 1)
visible_pc1 and 2 50176 50176
224 224 3
intersection1 43057
iouuuuuuuuu 0.8581194196428571
KEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEek
iou 43057
0.034531865
SCORE (0, 0, 0) (0, 0, 1) 102.615204
(1, 0, 0) (1, 0, 1)
visible_pc1 and 2 50176 50176
224 224 3
intersection1 43482
iouuuuuuuuu 0.8665896045918368
KEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEek
iou 43482
0.02552711
SCORE (1, 0, 0) (1, 0, 1) 56.647106
(2, 0, 0) (2, 0, 1)
visible_pc1 and 2 50176 50176
224 224 3
intersection1 18648
iouuuuuuuuu 0.3716517857142857
iou 18648
0.027871456
SCORE (2, 0, 0) (2, 0, 1) 28.959078
(3, 0, 0) (3, 0, 1)
visible_pc1 and 2 50176 50176
224 224 3
intersection1 9738
iouuuuuuuuu 0.19407684948979592
iou 9738
0.04760879
SCORE (3, 0, 0) (3, 0, 1) 44.08589
(4, 0, 0) (4, 0, 1)
visible_pc1 and 2 50176 50176
224 224 3
intersection1 46500
iouuuuuuuuu 0.9267378826530612
KEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEek
iou 46500
0.021765945
SCORE (4, 0, 0) (4, 0, 1) 44.047173
(5, 0, 0) (5, 0, 1)
visible_pc1 and 2 50176 50176


KeyboardInterrupt: 

In [ ]:
selected_pairs

In [60]:
for img1, img2 in selected_pairs:
    # print(img1[0])
    print(f"Selected pair: Image {img1[0]['idx']} and Image {img2[0]['idx']}")

In [2]:
from dust3r.datasets import UnderWaterDataset

In [3]:
dataset = UnderWaterDataset(split='test', ROOT='/home/aleksandra/dense_glomap_output', resolution=224)

In [9]:
type(dataset[0][0]['pts3d'])

numpy.ndarray

In [42]:
selected_pairs = select_best_pairs(dataset)
for img1, img2 in selected_pairs:
    print(f"Selected pair: Image {img1['idx']} and Image {img2['idx']}")

float32 float32


TypeError: unhashable type: 'numpy.ndarray'